# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
!pip install gdown
# download link 1
# !gdown --id '1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS' --output hw7_data.zip

# download link 2 (if above link failed)
# !gdown --id '1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7' --output hw7_data.zip

# download link 3 (if above link failed)
!gdown --id '1k2BfGrvhk8QRnr9Xvb04oPIKDr1uWFpa' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1k2BfGrvhk8QRnr9Xvb04oPIKDr1uWFpa
To: /kaggle/working/hw7_data.zip
100%|███████████████████████████████████████| 12.1M/12.1M [00:00<00:00, 169MB/s]
Archive:  hw7_data.zip
  inflating: hw7_train.json          
  inflating: hw7_test.json           
  inflating: hw7_dev.json            
  inflating: hw7_in-context-learning-examples.json  
Fri Sep 12 12:51:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Unc

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers>=4.21.0
!pip install accelerate==0.21.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer # Import AdamW directly from transformers

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
          torch.manual_seed(seed)
          if torch.cuda.is_available():
                    torch.cuda.manual_seed(seed)
                    torch.cuda.manual_seed_all(seed)
          np.random.seed(seed)
          random.seed(seed)
          torch.backends.cudnn.benchmark = False
          torch.backends.cudnn.deterministic = True
same_seeds(0)

## Load Model and Tokenizer






In [4]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

2025-09-12 12:52:58.116267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757681578.438719      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757681578.530176      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs:
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions

In [5]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [6]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [7]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 150

        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 150

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            ans_len = answer_end_token - answer_start_token + 1
            max_start = max(0, min(answer_start_token, len(tokenized_paragraph) - self.max_paragraph_len))
            min_start = max(0, min(answer_end_token - self.max_paragraph_len + 1, max_start))  # +1 防 end 邊界
            paragraph_start = random.randint(min_start, max_start) if min_start <= max_start else 0
            # 放寬: if end_index - start_index > 30: continue (pdf 沒限, 但實務 30 夠)

            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]

            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []

            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):

                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]

                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)

                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)

            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len

        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [8]:
def index_before_tokenize(tokens, start, end):
    char_count, new_start, new_end = 0, 512, 512
    start_flag = 0
    end_flag = 0
        
    for i, token in enumerate(tokens):
        if token == '[UNK]' or token == '[CLS]' or token == '[SEP]':
            if i == start:
                new_start = char_count
            if i == end:
                new_end = char_count
            char_count += 1
        else:
            for c in token:
                if i == start and start_flag == 0:
                    #print(token)
                    new_start = char_count
                    start_flag = 1
                if i == end:
                    #print(token)
                    new_end = char_count
                    end_flag = 1
                if c != '#':
                    char_count += 1
    return new_start, new_end

In [9]:
def evaluate(data, output, doc_stride=150, paragraph=None, paragraph_tokenized=None):
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]

    entire_start_index = 0
    entire_end_index = 0

    for k in range(num_of_windows):
        mask = (data[1][0][k].bool() & data[2][0][k].bool()).to(output.start_logits.device)

        masked_start_logits = torch.masked_select(output.start_logits[k], mask)[:-1]
        masked_end_logits = torch.masked_select(output.end_logits[k], mask)[:-1]

        if masked_start_logits.numel() == 0 or masked_end_logits.numel() == 0:
            continue

        start_prob, start_index = torch.max(masked_start_logits, dim=0)

        masked_end_after_start = masked_end_logits[start_index:]
        if masked_end_after_start.numel() == 0:
            continue

        end_prob, end_offset = torch.max(masked_end_after_start, dim=0)
        end_index = start_index + end_offset

        prob = start_prob + end_prob
        masked_data = torch.masked_select(
            data[0][0][k].to(output.start_logits.device), mask
        )[:-1]

        if prob > max_prob and 0 <= end_index - start_index <= 30:
            max_prob = prob
            entire_start_index = start_index.item() + doc_stride * k
            entire_end_index = end_index.item() + doc_stride * k
            answer = tokenizer.decode(masked_data[start_index:end_index + 1])
            answer = answer.replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e')\
                           .replace('☆','\u3000').replace('●','#').replace(' ','')

    if '[UNK]' in answer and paragraph is not None and paragraph_tokenized is not None:
        new_start, new_end = index_before_tokenize(tokens=paragraph_tokenized,
                                                   start=entire_start_index, end=entire_end_index)
        answer = paragraph[new_start:new_end+1]
        answer = answer.replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e')\
                       .replace('☆','\u3000').replace('●','#')

    return answer


## Training

In [10]:
from accelerate import Accelerator
import torch.optim as optim
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

num_epoch = 3
validation = True
logging_step = 100
learning_rate = 3e-5
doc_stride = 150
accum_iter = 4  # <<< 加這裡: pdf trick, 模擬大 batch (有效=8*4=32), 穩梯度
# batch accumulation parameter
# accum_iter = 4

optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)


fp16_training = True
if fp16_training:
    accelerator = Accelerator(mixed_precision="fp16")
else:
    accelerator = Accelerator()
    
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

# Total number of training steps
total_steps = len(train_loader) * num_epoch
print('total_steps', total_steps)
# Set up the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps= 0, # Default value
                                                num_training_steps=total_steps)

if not validation:
    dev_set = QA_Dataset("train", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
    train_set = torch.utils.data.ConcatDataset([train_set, dev_set])
    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for idx, data in enumerate(tqdm(train_loader)):
        # Load all data into GPU
        data = [i.to(device) for i in data]

        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])


        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)

        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss

        output.loss = output.loss / accum_iter  # <<< 加這裡: normalize loss for accumulation

        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()

        if ((idx + 1) % accum_iter == 0) or (idx + 1 == len(train_loader)):  # <<< 加這裡: 累積後 update
            accelerator.clip_grad_norm_(model.parameters(), max_norm=1.0)  # <<< 加這裡: pdf 隱含, 防梯度爆炸
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        step += 1

        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, doc_stride, dev_paragraphs[dev_questions[i]['paragraph_id']],
                    dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].tokens) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model"
model.save_pretrained(model_save_dir)

total_steps 10095
Start Training ...


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:427: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:1301: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  new_forward = torch.cuda.amp.autocast(dtype=torch.float16)(model_forward_func)


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 3.772, acc = 0.090
Epoch 1 | Step 200 | loss = 1.835, acc = 0.356
Epoch 1 | Step 300 | loss = 1.293, acc = 0.520
Epoch 1 | Step 400 | loss = 1.036, acc = 0.602
Epoch 1 | Step 500 | loss = 1.006, acc = 0.599
Epoch 1 | Step 600 | loss = 1.052, acc = 0.591
Epoch 1 | Step 700 | loss = 0.929, acc = 0.637
Epoch 1 | Step 800 | loss = 0.860, acc = 0.674
Epoch 1 | Step 900 | loss = 0.882, acc = 0.660
Epoch 1 | Step 1000 | loss = 0.848, acc = 0.680
Epoch 1 | Step 1100 | loss = 0.761, acc = 0.699
Epoch 1 | Step 1200 | loss = 0.867, acc = 0.667
Epoch 1 | Step 1300 | loss = 0.819, acc = 0.675
Epoch 1 | Step 1400 | loss = 0.815, acc = 0.671
Epoch 1 | Step 1500 | loss = 0.742, acc = 0.689
Epoch 1 | Step 1600 | loss = 0.807, acc = 0.674
Epoch 1 | Step 1700 | loss = 0.827, acc = 0.691
Epoch 1 | Step 1800 | loss = 0.789, acc = 0.692
Epoch 1 | Step 1900 | loss = 0.785, acc = 0.695
Epoch 1 | Step 2000 | loss = 0.773, acc = 0.712
Epoch 1 | Step 2100 | loss = 0.782, acc = 0.688
E

  0%|          | 0/2863 [00:00<?, ?it/s]

...

Validation | Epoch 2 | acc = 0.805


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 3 | Step 100 | loss = 0.379, acc = 0.826
Epoch 3 | Step 200 | loss = 0.378, acc = 0.840
Epoch 3 | Step 300 | loss = 0.316, acc = 0.854
Epoch 3 | Step 400 | loss = 0.380, acc = 0.814
Epoch 3 | Step 500 | loss = 0.372, acc = 0.842
Epoch 3 | Step 600 | loss = 0.325, acc = 0.847
Epoch 3 | Step 700 | loss = 0.366, acc = 0.821
Epoch 3 | Step 800 | loss = 0.392, acc = 0.835
Epoch 3 | Step 900 | loss = 0.366, acc = 0.815
Epoch 3 | Step 1000 | loss = 0.363, acc = 0.851
Epoch 3 | Step 1100 | loss = 0.352, acc = 0.840
Epoch 3 | Step 1200 | loss = 0.363, acc = 0.831
Epoch 3 | Step 1300 | loss = 0.416, acc = 0.821
Epoch 3 | Step 1400 | loss = 0.362, acc = 0.826
Epoch 3 | Step 1500 | loss = 0.356, acc = 0.830
Epoch 3 | Step 1600 | loss = 0.424, acc = 0.814
Epoch 3 | Step 1700 | loss = 0.330, acc = 0.857
Epoch 3 | Step 1800 | loss = 0.351, acc = 0.835
Epoch 3 | Step 1900 | loss = 0.341, acc = 0.819
Epoch 3 | Step 2000 | loss = 0.327, acc = 0.842
Epoch 3 | Step 2100 | loss = 0.321, acc = 0.846
E

  0%|          | 0/2863 [00:00<?, ?it/s]

Validation | Epoch 3 | acc = 0.834
Saving Model ...


## Testing

In [11]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, doc_stride, test_paragraphs[test_questions[i]['paragraph_id']],
                               test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "macbert.csv"
with open(result_file, 'w') as f:
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Completed! Result is in macbert.csv
